In [4]:
# Create config column
import pandas as pd
df_by_config = pd.read_pickle('../output/df_metrics')
df_by_config['config'] = df_by_config["in_len"].astype(str) + '_' + df_by_config["out_len"].astype(str)
df_by_config.drop(columns=['in_len','out_len','iteration'])
df_by_config = df_by_config.groupby(['model','checkpoint','config']).mean().reset_index()
df_by_config.head()

             model                                     checkpoint config  \
0       melody_rnn                                  attention_rnn    1_1   
1       melody_rnn                                      basic_rnn    1_1   
2       melody_rnn                                   lookback_rnn    1_1   
3       melody_rnn                                       mono_rnn    1_1   
4  performance_rnn  density_conditioned_performance_with_dynamics    1_1   

   iteration        time  in_len  out_len  bar_pitch_class_histogram_kl_div  \
0          1   45.081079       1        1                          0.883171   
1          1   64.063910       1        1                          0.883171   
2          1   64.356445       1        1                          0.883171   
3          1  106.742966       1        1                          0.883171   
4          1   10.111051       1        1                          0.608409   

   bar_pitch_class_histogram_overlap  bar_used_pitch_kl_div  ...  \


In [18]:
# Get Subset DFs
col_metrics = df_by_config.columns
col_kl_divs  = [x for x in col_metrics if x.endswith('kl_div')]
col_overlaps = [x for x in col_metrics if x.endswith('overlap')]

# Calculate KL Divs mean and std per model config
loc_kldivs = df_by_config[['model'] + col_kl_divs].loc[: , "bar_pitch_class_histogram_kl_div":"note_length_hist_kl_div"]
df_by_config['kl_mean'] = loc_kldivs.mean(axis=1)
df_by_config['kl_std'] = loc_kldivs.std(axis=1)
loc_overlaps = df_by_config[['model'] + col_overlaps].loc[: , "bar_pitch_class_histogram_overlap":"note_length_hist_overlap"]
df_by_config['overlap_mean'] = loc_overlaps.mean(axis=1)
df_by_config['overlap_std'] = loc_overlaps.std(axis=1)
df_mean_metrics = df_by_config[['model','checkpoint','config','overlap_mean','overlap_std','kl_mean','kl_std']].copy()


# Print
print(df_mean_metrics)

             model                                     checkpoint config  \
0       melody_rnn                                  attention_rnn    1_1   
1       melody_rnn                                      basic_rnn    1_1   
2       melody_rnn                                   lookback_rnn    1_1   
3       melody_rnn                                       mono_rnn    1_1   
4  performance_rnn  density_conditioned_performance_with_dynamics    1_1   

   overlap_mean  overlap_std   kl_mean    kl_std  
0      0.407962     0.269273  0.198837  0.237318  
1      0.407962     0.269273  0.198837  0.237318  
2      0.407962     0.269273  0.198837  0.237318  
3      0.407962     0.269273  0.198837  0.237318  
4      0.531119     0.322589  0.213528  0.176577  